In [3]:
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render

from stackable_pipe import adapter, socket, ring
from solid import cube, rotate, translate

props = dict(d=60, w=4)
scene = rotate((0,0,60))(adapter(**props))
scene += rotate((0,0,0))(translate((0,0,props['w']))(socket(**props) ))
scene -= cube(100)

render(socket(**props), outfile='test_socket_exact_fit.stl')


In [2]:
from stackable_pipe import pipe, slot, ring, socket
from solid import cylinder, translate, rotate

w = 4
h = 160
d=120
pipe_count = 4

def spin_around_tower(obj, count=pipe_count):
    angles = range(0,360, int(360/count))
    return translate((0,0,h/2))( [rotate((90,0,z))(obj) for z in angles])

def tower_socket(d=60, w=4):
    r = d/2
    base = rotate((0,0,15))(socket(d=d, w=w))
    transition = ring(r2=r+w*3, r1=r, h=w*3, w=w*4) - cylinder(h=w*3, d=d )

    return base + transition


tower = pipe(d=d, h = h, w=w)
outline = cylinder(d=d, h=h, segments=60)
holes = spin_around_tower(cylinder(d=60, h=d, segments=60))
sockets = spin_around_tower(translate((0,0,(d/2)-(3*w)))(tower_socket()))

hydro_tower = tower - holes 
hydro_tower += sockets
# hydro_tower *= outline

# render(hydro_tower, outfile='hydro_tower.stl')

In [3]:
from stackable_pipe import pipe, slot, ring, socket, adapter
from solid import cylinder, translate, rotate, cube

w=4
d = 60

bottom = adapter(d=d, w=w)
top = translate((0,0,4*w))(cylinder(d=d*1.2,h=5, segments=6))

hydro_tower_cap = top + bottom
hydro_tower_cap *= cube((d*2, d*2, 10*w), center=True)

# render(hydro_tower_cap, outfile='hydro_tower_cap.stl')

In [4]:
from stackable_pipe import pipe, slot, ring, socket, adapter
from solid import cylinder, translate, rotate, cube, hull

w=4
d = 60

def angle_cylinder(d=d, d2=d):
    d2 = d2 or d
    cyl = cylinder(d=d, h=2) + translate((d2/2,0,d2/4*3))(rotate((0,45,0))(cylinder(d=d, h=2)))
    cyl = hull()(cyl)
    
    return cyl

def angle_pipe(d=d, w=w):
    tube = angle_cylinder(d=d)
    hole = angle_cylinder(d=d-2*w, d2=d+w)
    
    return tube - hole

def adapter_bottom_grid(d=d, w=w):
    plate = cylinder(h=2, d=d-4*w)
    hole = cylinder(h=2, d=6)
    
    holes = sum([rotate((0,0,r))(translate((x,0,0))(hole)) for r in range(0,360,360//8) for x in [0, 9, 16]])

    return plate - holes

bottom = adapter(d=d, w=w)
grid = adapter_bottom_grid()
pipe = angle_pipe(d=d, w=w/2)

hydro_tower_plant_cup =  bottom  +grid +  translate((0,0,7*w))(pipe)

# render(hydro_tower_plant_cup, outfile='hydro_tower_plant_cup.stl')

In [5]:
d = 120
w=4
caps =  spin_around_tower(translate((0,0, d/2- 4*w))(hydro_tower_cap))

render(hydro_tower + caps - cube(200))